# N body problem

According to Newton's law a force of attraction between body $i$ and $j$ can be expressed as

$$
\mathbf{F}_{ij}
    = \frac{G m_i m_j}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\| ^ 2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

Resultant force acting on $i$ can be found when summing over all the interactions with $N-1$ bodies 

$$
\mathbf{F}_{i}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_i m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

or in explicit differential form

$$
\frac{d^2\mathbf{r}_i}{dt^2}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$




In [43]:
import numpy as np
from typing import List, Dict, Tuple, Self
from itertools import chain, permutations

G = 1

In [44]:
class Body:
    def __init__(
        self,
        m : float,
        r_0 : Tuple[float, float], 
        v_0 : Tuple[float, float],
        name: str = "body"
    ) -> None:
        self.m = m
        self.r_0 = r_0
        self.x_0 = r_0[0]
        self.y_0 = r_0[1]
        self.v_0 = v_0
        self.vx_0 = v_0[0]
        self.vy_0 = v_0[1]
        self.name = name

    def distance(self, body: Self) -> float:
        return np.sqrt((self.x_0 - body.x_0) ** 2 + (self.y_0 - body.y_0) ** 2)

earth = Body(
    m = 1, 
    r_0 = (1, 0), 
    v_0 = (0, np.sqrt(333000)),
    name = "earth"
)
sun = Body(
    m = 333000, 
    r_0 = (0, 0),
    v_0 = (0, 0),
    name = "sun"
)

r_sun_earth = sun.distance(earth)
r_sun_earth

1.0

In [51]:
class System:
    def __init__(self, *bodies: Body) -> None:
        self.bodies = bodies
        self.bodies_dict = {body.name : body for body in bodies}

    def prepare_system_vector(self):
        """
        s = [
            body[0]_positions, body[1]_positions,
            body[0]_v, body[1]_v,
        ]
        """
        positions = [[body.x_0, body.y_0] for body in self.bodies]
        velocities = [[body.vx_0, body.vy_0] for body in self.bodies]
        system_vector = list(chain(*positions+velocities))

        velocities = [body.v_0 for body in self.bodies]
        reduced_rhs_list = [body for body in self.bodies]

        dsdt = velocities + reduced_rhs_list
        return list(chain(*dsdt))
    
    @staticmethod
    def get_reduced_rhs(body_1 : Body, body_2 : Body):
        """
        calculates m_2 * (r_2 - r_1) / |r_2 - r_1| ^ 3
        """
        return [
            body_2.m / body_1.distance(body_2) ** 3 * (body_2.r_0[dimension] - body_1.r_0[dimension])
            for dimension 
            in [0,1]
        ]
# test           
s = System(earth, sun)
s.bodies

s.bodies_dict

s.prepare_system_vector()

#s.get_reduced_rhs(earth, sun)




[0, 577.0615218501404, 0, 0]